In [130]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')
import torch as tr 
import numpy as np 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [153]:
from src.lda_model import LDASmoothed 
from src.utils import get_vocab_from_docs, get_np_wct 
from pprint import pprint 

import copy 

In [138]:
num_topics = 3 

In [149]:
docs = [
    ['My name is tuo tuo .'],
    ['tuo tuo is cute !'],
    ['Tuo tuo likes to poo a lot !'],
    ['mama bear very tuo !']
]

docs = [doc[0].split(' ') for doc in docs]

print(docs)

[['My', 'name', 'is', 'tuo', 'tuo', '.'], ['tuo', 'tuo', 'is', 'cute', '!'], ['Tuo', 'tuo', 'likes', 'to', 'poo', 'a', 'lot', '!'], ['mama', 'bear', 'very', 'tuo', '!']]


In [150]:
wct_dict = get_vocab_from_docs(docs)
wct_dict 

{'My': [1, 0, 0, 0],
 'name': [1, 0, 0, 0],
 'is': [1, 1, 0, 0],
 'tuo': [2, 2, 1, 1],
 '.': [1, 0, 0, 0],
 'cute': [0, 1, 0, 0],
 '!': [0, 1, 1, 1],
 'Tuo': [0, 0, 1, 0],
 'likes': [0, 0, 1, 0],
 'to': [0, 0, 1, 0],
 'poo': [0, 0, 1, 0],
 'a': [0, 0, 1, 0],
 'lot': [0, 0, 1, 0],
 'mama': [0, 0, 0, 1],
 'bear': [0, 0, 0, 1],
 'very': [0, 0, 0, 1]}

In [151]:
wct_array, w_2_idx = get_np_wct(wct_dict, docs)

print(wct_array)
pprint(w_2_idx)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [2. 2. 1. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
{'!': 6,
 '.': 4,
 'My': 0,
 'Tuo': 7,
 'a': 11,
 'bear': 14,
 'cute': 5,
 'is': 2,
 'likes': 8,
 'lot': 12,
 'mama': 13,
 'name': 1,
 'poo': 10,
 'to': 9,
 'tuo': 3,
 'very': 15}


In [152]:
wct_array.astype(object)

array([[1.0, 0.0, 0.0, 0.0],
       [1.0, 0.0, 0.0, 0.0],
       [1.0, 1.0, 0.0, 0.0],
       [2.0, 2.0, 1.0, 1.0],
       [1.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0.0],
       [0.0, 1.0, 1.0, 1.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 1.0],
       [0.0, 0.0, 0.0, 1.0],
       [0.0, 0.0, 0.0, 1.0]], dtype=object)

In [155]:
docs_in_vocab_indx = copy.deepcopy(docs)
#print(docs_in_vocab_indx)

for id, d in enumerate(docs): 

    for n in range(len(d)): 

        docs_in_vocab_indx[id][n] = w_2_idx[docs[id][n]]

print(docs_in_vocab_indx)
print(docs)
# docs_in_idx = np.array(docs_in_vocab_indx)
# docs_in_idx

[[0, 1, 2, 3, 3, 4], [3, 3, 2, 5, 6], [7, 3, 8, 9, 10, 11, 12, 6], [13, 14, 15, 3, 6]]
[['My', 'name', 'is', 'tuo', 'tuo', '.'], ['tuo', 'tuo', 'is', 'cute', '!'], ['Tuo', 'tuo', 'likes', 'to', 'poo', 'a', 'lot', '!'], ['mama', 'bear', 'very', 'tuo', '!']]


- construct  new phi 

In [161]:
docs  

[['My', 'name', 'is', 'tuo', 'tuo', '.'],
 ['tuo', 'tuo', 'is', 'cute', '!'],
 ['Tuo', 'tuo', 'likes', 'to', 'poo', 'a', 'lot', '!'],
 ['mama', 'bear', 'very', 'tuo', '!']]

In [157]:
phi = tr.zeros(len(docs),wct_array.shape[0],num_topics)

for id, d in enumerate(docs): 

    for word in d: 

        v = w_2_idx[word]

        phi[id][v] = 1/num_topics 

phi 

tensor([[[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 

In [187]:
lda = LDASmoothed(
    docs,
    num_topics, 
    wct_dict, 
    word_ct_array = wct_array
)
#LDASmoothed(docs, 3, wct_dict)

Word mapping to vocab index is: 
{'!': 6,
 '.': 4,
 'My': 0,
 'Tuo': 7,
 'a': 11,
 'bear': 14,
 'cute': 5,
 'is': 2,
 'likes': 8,
 'lot': 12,
 'mama': 13,
 'name': 1,
 'poo': 10,
 'to': 9,
 'tuo': 3,
 'very': 15}
Topic Dirichlet Prior, Alpha
torch.Size([3])
tensor([0.8556, 0.8763, 0.8933], dtype=torch.float64)

Word Dirichlet Prior, Eta
torch.Size([16])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)

Var Inf - Word Dirichlet prior, Lambda
torch.Size([3, 16])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
       dtype=torch.float64)

Var Inf - Topic Dirichlet prior, Gamma
torch.Size([4, 3])
tensor([[6.1890, 6.2096, 6.2267],
        [6.1890, 6.2096, 6.2267],
        [6.1890, 6.2096, 6.2267],
        [6.1890, 6.2096, 6.2267]], dtype=torch.float64)

Var -Inf - Wor

In [189]:
lda.e_step()

Iteration 1, Delta Gamma = inf
Iteration 2, Delta Gamma = 0.04562150804439216
Iteration 3, Delta Gamma = 0.04846251080266142
Iteration 4, Delta Gamma = 0.05261327520621401
Iteration 5, Delta Gamma = 0.058070423364323824
Iteration 6, Delta Gamma = 0.06495961380507033
Iteration 7, Delta Gamma = 0.07349271877618255
Iteration 8, Delta Gamma = 0.08395050557190564
Iteration 9, Delta Gamma = 0.0966775787399271
Iteration 10, Delta Gamma = 0.11208407796266077
Iteration 11, Delta Gamma = 0.13065081497116196
Iteration 12, Delta Gamma = 0.15293323552689791
Iteration 13, Delta Gamma = 0.17955355629076167
Iteration 14, Delta Gamma = 0.2111563436105005
Iteration 15, Delta Gamma = 0.24827729152007688
Iteration 16, Delta Gamma = 0.29104160025234455
Iteration 17, Delta Gamma = 0.33859389554914476
Iteration 18, Delta Gamma = 0.38823704778309215
Iteration 19, Delta Gamma = 0.43453081927449044
Iteration 20, Delta Gamma = 0.4691031970009005
Iteration 21, Delta Gamma = 0.4823231416594282
Iteration 22, Delta 

In [173]:
round(tr.sum(tr.tensor([0.3333,0.3333,0.3333])).item())

1

In [101]:
lda._phi_[:][2][-1] = [1,1,1]

In [102]:
lda._phi_

array([array([[0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333]]),
       array([[0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333]]),
       array([[0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [0.33333333, 0.33333333, 0.33333333],
              [1.        , 1.        , 1.        ]]),
      

In [104]:
lda._phi_[:][2][-1].dtype

dtype('float64')

In [115]:
x = tr.tensor(lda._phi_[:][2][-1])
x1 = x.clone()

In [116]:
x1[1]= 1e8

In [117]:
x1 

tensor([1.0000e+00, 1.0000e+08, 1.0000e+00], dtype=torch.float64)

In [118]:
x

tensor([1., 1., 1.], dtype=torch.float64)